In [1]:
import pandas as pd

In [4]:
ls

 Volume in drive G is Tiger
 Volume Serial Number is 3EB1-C7DB

 Directory of G:\CU Boulder\Summer 2023\data mining\final_project

06/22/2023  01:33 PM    <DIR>          .
06/22/2023  01:33 PM    <DIR>          ..
06/22/2023  01:31 PM    <DIR>          .ipynb_checkpoints
06/22/2023  01:30 PM        10,860,387 data.csv
06/22/2023  01:33 PM            13,685 final_project.ipynb
               2 File(s)     10,874,072 bytes
               3 Dir(s)  758,847,909,888 bytes free


In [13]:
data = pd.read_csv('./data.csv')

In [14]:
data.head()

,professor_name,school_name,department_name,local_name,state_name,year_since_first_review,star_rating,take_again,diff_index,tag_professor,...,lots_of_homework,accessible_outside_class,lecture_heavy,extra_credit,graded_by_few_things,group_projects,test_heavy,so_many_papers,beware_of_pop_quizzes,IsCourseOnline
0,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,...,0,0,0,0,0,1,0,0,0,0
1,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,...,0,0,0,0,0,1,0,0,0,0
2,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,...,0,0,0,0,0,1,0,0,0,0
3,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,...,0,0,0,0,0,1,0,0,0,0
4,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,...,0,0,0,0,0,1,0,0,0,0


In [21]:
data[data['local_name'] == ' CO']

,professor_name,school_name,department_name,local_name,state_name,year_since_first_review,star_rating,take_again,diff_index,tag_professor,...,lots_of_homework,accessible_outside_class,lecture_heavy,extra_credit,graded_by_few_things,group_projects,test_heavy,so_many_papers,beware_of_pop_quizzes,IsCourseOnline
14265,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14266,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14267,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14268,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14269,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14270,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14271,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14272,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14273,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14274,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0


In [25]:
data[data['local_name'] == ' Denver']

,professor_name,school_name,department_name,local_name,state_name,year_since_first_review,star_rating,take_again,diff_index,tag_professor,...,lots_of_homework,accessible_outside_class,lecture_heavy,extra_credit,graded_by_few_things,group_projects,test_heavy,so_many_papers,beware_of_pop_quizzes,IsCourseOnline
14265,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14266,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14267,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14268,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14269,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14270,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14271,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14272,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14273,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
14274,Rudi Hartmann,University of Colorado Denver,Geography department,Denver,CO,14.0,2.8,NaN,2.3,Participation matters (3) Get ready to read (...,...,0,0,1,0,0,0,0,0,0,0
